In [1]:
#Importing necessary Libraries
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
import os
import keras

In [2]:
# There are two classes of interests among the images in dataset
cls_of_int = ['Covid', 'Normal']

In [3]:
#setting the path variables for train, test and validation data
basePath = "Data"
trainPath = os.path.join(basePath,"train")
testPath = os.path.join(basePath,"test")
validPath = os.path.join(basePath,"valid")

trainCovid = os.path.join(trainPath,"Covid")
trainNormal = os.path.join(trainPath,"Normal")

testCovid = os.path.join(testPath,"Covid")
testNormal = os.path.join(testPath,"Normal")

validCovid = os.path.join(basePath,"valid","Covid")
validNormal = os.path.join(basePath,"valid","Normal")

In [4]:
# using a data generator for data augmentation, the volume of training data is increased using data augmentation techniques
train_data_generator = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1.0/255, # Normalizing the image
    shear_range=0.2,
    zoom_range=0.2,
    vertical_flip = False,
    fill_mode='nearest',
    featurewise_center = True,
    featurewise_std_normalization = True)

In [5]:
# The data generator for validation data, normalizing of the image is done
valid_data_generator = ImageDataGenerator(rescale = 1./255)

In [6]:
# The data generator for test data, normalizing of the image is done
test_data_generator = ImageDataGenerator(rescale = 1./255)

In [7]:
# Read data from directory for training data
train_generator = train_data_generator.flow_from_directory(
    directory=trainPath,
    target_size=(224, 224), # resizing the image
    color_mode="rgb",
    batch_size=32,
    class_mode="binary", # class mode is binary as there are only 2 classes
    shuffle=True,
    seed=42
)

Found 11047 images belonging to 2 classes.


In [8]:
# Read data from directory for validation data
valid_generator = valid_data_generator.flow_from_directory(
    directory= validPath,
    target_size=(224, 224), # resizing the image
    color_mode="rgb",
    batch_size=32,
    class_mode="binary", # class mode is binary as there are only 2 classes
    shuffle=True,
    seed=42
)

Found 2208 images belonging to 2 classes.


In [9]:
# Read data from directory for test data
test_generator = test_data_generator.flow_from_directory(
    directory= testPath,
    target_size=(224, 224), # resizing the image
    color_mode="rgb",
    batch_size=11,
    class_mode='binary', # class mode is binary as there are only 2 classes
    shuffle=False,
    seed=42
)

Found 2761 images belonging to 2 classes.


In [10]:
# Downloading the inception V3 model.
pretrained_model = tf.keras.applications.InceptionV3(
        weights='imagenet',
        include_top=False ,
        input_shape=(224,224,3)
    )
pretrained_model.trainable = False

# Added Pooling layer and a sigmoid activation layer.    
model = tf.keras.Sequential([ 
        pretrained_model,   
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

87916544/87910968 [==============================] - 14s 0us/step


In [11]:
# Compiling the model with Adam optimizer and binary cross entropy loss function
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer = opt ,
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [12]:
# To display the summary of the model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 8, 8, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 4098      
Total params: 21,806,882
Trainable params: 4,098
Non-trainable params: 21,802,784
_________________________________________________________________


In [13]:
# specifying the batch sizes
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

In [14]:
# Fitting the model with 20 epochs
model.fit_generator(generator = train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=20
)

G:\Users\Ajay\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
G:\Users\Ajay\anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
G:\Users\Ajay\anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/20
345/345 [==============================] - 710s 2s/step - loss: 0.4353 - accuracy: 0.8170 - val_loss: 0.3160 - val_accuracy: 0.8700
Epoch 2/20
345/345 [==============================] - 642s 2s/step - loss: 0.3062 - accuracy: 0.8782 - val_loss: 0.2960 - val_accuracy: 0.8759
Epoch 3/20
345/345 [==============================] - 638s 2s/step - loss: 0.2962 - accuracy: 0.8794 - val_loss: 0.2847 - val_accuracy: 0.8845
Epoch 4/20
345/345 [==============================] - 654s 2s/step - loss: 0.2824 - accuracy: 0.8874 - val_loss: 0.3009 - val_accuracy: 0.8759
Epoch 5/20
345/345 [==============================] - 639s 2s/step - loss: 0.2774 - accuracy: 0.8900 - val_loss: 0.2727 - val_accuracy: 0.8963
Epoch 6/20
345/345 [==============================] - 616s 2s/step - loss: 0.2756 - accuracy: 0.8892 - val_loss: 0.2748 - val_accuracy: 0.8949
Epoch 7/20
345/345 [==============================] - 640s 2s/step - loss: 0.2628 - accuracy: 0.8944 - val_loss: 0.2695 - val_accuracy: 0.8895

In [15]:
model.evaluate(valid_generator,steps=STEP_SIZE_VALID)

69/69 [==============================] - 92s 1s/step - loss: 0.2495 - accuracy: 0.8958


[0.24947570264339447, 0.8958333134651184]

In [16]:
model.evaluate(test_generator)

251/251 [==============================] - 121s 482ms/step - loss: 0.2609 - accuracy: 0.8950


[0.2608718276023865, 0.8949655890464783]

In [17]:
# Evaluating the validation data set
score = model.evaluate(valid_generator,batch_size=32)
print("Accuracy: {:.2f}%".format(score[1] * 100)) 
print("Loss: ",score[0])

69/69 [==============================] - 90s 1s/step - loss: 0.2495 - accuracy: 0.8958
Accuracy: 89.58%
Loss:  0.2494756430387497


In [18]:
# Evaluating the test data set
score = model.evaluate(test_generator,batch_size=11)
print("Accuracy: {:.2f}%".format(score[1] * 100)) 
print("Loss: ",score[0])

251/251 [==============================] - 122s 485ms/step - loss: 0.2609 - accuracy: 0.8950
Accuracy: 89.50%
Loss:  0.2608718276023865


In [19]:
model.save('imagegeneratormodel')

INFO:tensorflow:Assets written to: imagegeneratormodel\assets


In [20]:
loaded_model = keras.models.load_model('imagegeneratormodel')

In [21]:
valscore = loaded_model.evaluate(valid_generator)
print("Accuracy: {:.2f}%".format(valscore[1] * 100))

69/69 [==============================] - 100s 1s/step - loss: 0.2495 - accuracy: 0.8958
Accuracy: 89.58%


In [22]:
score = loaded_model.evaluate(test_generator)
print("Accuracy: {:.2f}%".format(score[1] * 100))

251/251 [==============================] - 120s 478ms/step - loss: 0.2609 - accuracy: 0.8950
Accuracy: 89.50%
